
<h1> Information Regarding the project </h1>
<p> <h3>This project is designed to classify wildlife species </h3>
 Coded by: - Mainuddin Alam Irteja
</p>

In [12]:
# Connecting to my google drive where the image set is kept
# Image set was used from kaggle
# Link to the image dataset:- https://www.kaggle.com/datasets/vishweshsalodkar/wild-animals/data
import os
from google.colab import drive
drive.mount('/content/gdrive')
rootDir = "/content/gdrive/MyDrive/Colab Notebooks/Wildlife Identifier/Animals"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
# Required modules for the project
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
# Loading and Augmenting the images
trainDir = os.path.join(rootDir, 'train')
validationDir= os.path.join(rootDir, 'validation')
testDir = os.path.join(rootDir, 'testing')

# Implementing data augmentation for the training set
trainDatagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Rescale validation and test sets
validationDatagen = ImageDataGenerator(rescale=1./255)
testDatagen = ImageDataGenerator(rescale=1./255)

# Create data generators
trainGenerator = trainDatagen.flow_from_directory(
    trainDir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validationGenerator = validationDatagen.flow_from_directory(
    validationDir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

testGenerator = testDatagen.flow_from_directory(
    testDir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 118 images belonging to 5 classes.
Found 32 images belonging to 5 classes.
Found 20 images belonging to 5 classes.


In [ ]:
#